**Find the ground state energy of a certain Hamiltonian with SDP:**
1. We can somehow prepare the groud state $\rho_g$ of a Hamiltonian $H$
2. We do quantum tomography on this prepared state $\rho_g$ and get an approximation $\hat{\rho}$
3. By using SDP, we get a physically valid quantum state $\hat{\rho}_{SDP}$ which minimizes $\text{Tr}(\rho H)$ for $\rho \in \tilde{\rho}$
4. The ground state energy we find is then $\text{Tr}(H\hat{\rho}_{SDP})$

In [75]:
import argparse
import time
import random
import itertools
import numpy as np
import cvxpy as cp
import math
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from qutip import *
from qiskit import *
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, DensityMatrix, Operator, Pauli, partial_trace, state_fidelity, random_density_matrix
from qiskit.visualization import plot_histogram, plot_state_city, plot_bloch_multivector, plot_state_paulivec, plot_state_hinton, plot_state_qsphere
from qiskit.tools.monitor import job_monitor
import os
#os.environ["OMP_NUM_THREADS"] = "10"

def tenToAny(origin, N, n):
    # 10进制转换为n进制list
    list = []
    while True:
        s = origin // n
        tmp = origin % n
        list.append(tmp)
        if s == 0:
            break
        origin = s
    list.reverse()
    list = [str(each) for each in list]
    while len(list) < N:
        list.insert(0, '0')
    return list
def generate_PauliStrList(N):
    ''' Given the number of qubits N, return its corresponding Pauli vector.
    E.g.:
        INPUT: N=2
        OUTPUT: ['II','IX',...'ZZ']
    '''
    Pauli_str_list = []
    for i in range(4 ** N):
        pauli_num_list = tenToAny(i, N, 4)
        pauli_basis_list = pauli_num_list
        for j in range(N):
            if pauli_num_list[j] == '0':
                pauli_basis_list[j] = 'I'
            elif pauli_num_list[j] == '1':
                pauli_basis_list[j] = 'X'
            elif pauli_num_list[j] == '2':
                pauli_basis_list[j] = 'Y'
            else:
                pauli_basis_list[j] = 'Z'
        Pauli_str_list.append(''.join(pauli_basis_list))

    return Pauli_str_list
def generate_sub_PauliStrList(PauliStrList, index):
    # Stupid version
    ''' Given a index (list) of qubits, retrun the Pauli vectors of this sub system.
    E.g.:
        INPUT: PauliStrList=['III',...'ZZZ'], index=[0,2]
        OUTPUT: ['III','IIX','IIY','IIZ','XII','XIX',...'ZIZ']
    '''
    output = list()
    no_meas = list(set(list(range(N))) - set(index))
    for i in PauliStrList:
        trigger = bool(1)
        for j in no_meas:
            trigger = bool(trigger and i[int(j)] == 'I')
        if trigger: output.append(i)

    return output
def generate_sub_PauliStrList(N, index_list):
    # Less-complexity version
    base_string = 'I' * N
    output_strings = []

    for combination in itertools.product('IXYZ', repeat=len(index_list)):
        if all(c == 'I' for c in combination):
            continue

        temp_string = list(base_string)
        for index, char in zip(index_list, combination):
            temp_string[index] = char

        output_strings.append(''.join(temp_string))

    return output_strings
def pauliToMatrix(pauli_str):
    '''Given a Pauli string basis (str),
       output its corresponding matrix representation (Qobj data).
    '''
    pauli_basis_list = list()
    for basis in pauli_str:
        if basis == 'I':
            pauli_basis_list.append(qeye(2))
        elif basis == 'X':
            pauli_basis_list.append(sigmax())
        elif basis == 'Y':
            pauli_basis_list.append(sigmay())
        else:
            pauli_basis_list.append(sigmaz())
    return tensor(pauli_basis_list)

def Hamiltonian_matrix(H, H_sign_list):
    '''Given a list of Pauli string for each subsystem,
       output a list of their matrix representation.
    '''
    Hamiltonian_matrix = 0
    for i in range(len(H)):
        Hamiltonian_matrix = Hamiltonian_matrix + H_sign_list[i]*(pauliToMatrix(H[i]))
    return Hamiltonian_matrix
def Hamiltonian_global(H_local_list, N, M, K):
    '''Given the Hamiltonian of local subsystem (list of Pauli strings)
       return the Hamiltonian of global system (list of Pauli strings)
    '''
    H_global = []
    for i in range(K):
        for h in H_local_list:
            H_global.append(i * 'I' + h + (N - M - i) * 'I')
    return H_global
def ground_state(H_matrix):
    '''Given a matrix representation of a Hamiltonian,
       find the ground state energy, i.e. the minimum eigenvalue of the matrix,
       and the ground state density matrix
    '''
    H_matrix = np.array(H_matrix)
    eigenvalue, eigenvector = np.linalg.eigh(H_matrix)

    tmp = np.argsort(eigenvalue)
    ground_state_energy = eigenvalue[tmp[0]]
    ground_state_vec = np.array(eigenvector[:, tmp[0]])

    ground_state_dm = np.outer(ground_state_vec, np.conj(ground_state_vec))

    return ground_state_energy, ground_state_dm
def N_meas_list_func(start, end, num):
    '''Generate a list of number of measurement for the loop
    '''
    a = pow(end / start, 1 / (num - 1))
    N_meas_list = [start]
    for i in range(num - 1):
        N_meas_list.append(math.floor(a * N_meas_list[-1]))

    return N_meas_list
def gs_energy_estimate(measurement_dataset, confidence_level, H_global_list):
    '''Given the Pauli decomposition of the Hamiltonian of interest and measurement dataset
       return the expectaion value of the Hamiltonian (with confidence interval)
    '''
    E_min = 0
    E_max = 0
    error_rate = 1 - confidence_level
    z = norm.ppf(1 - error_rate / 2)  # Quantile of the input confidence level for binomial distribution
    
    for pauli_basis_str in H_global_list:
        exp, var = exp_var_calculator(measurement_dataset, pauli_basis_str)
        num_meas_sub = num_meas_sub_calculator(measurement_dataset, pauli_basis_str)
        p_value = 0.5 * (1 + exp)
        sigma = z * ((p_value * (1 - p_value) / num_meas_sub) ** 0.5)
        E_min = E_min + exp - 2*sigma
        E_max = E_max + exp + 2*sigma

    return E_min, E_max
def gs_energy_estimate(measurement_dataset, confidence_level, H_global_list):
    '''Given the Pauli decomposition of the Hamiltonian of interest and measurement dataset
       return the expectaion value of the Hamiltonian (with confidence interval)
       This version is more rigorous.
    '''
    E_sum = 0
    var_sum = 0
    error_rate = 1 - confidence_level
    z = norm.ppf(1 - error_rate / 2)  # Quantile of the input confidence level for binomial distribution
    
    for pauli_basis_str in H_global_list:
        exp, var = exp_var_calculator(measurement_dataset, pauli_basis_str)
        num_meas_sub = num_meas_sub_calculator(measurement_dataset, pauli_basis_str)
        E_sum = E_sum + exp
        var_sum = var_sum + var/num_meas_sub

    E_min = E_sum - 2.58*var_sum**0.5
    E_max = E_sum + 2.58*var_sum**0.5

    return E_min, E_max
def lower_bound_with_SDP(H, N, M, K, P):
    '''Solve the SDP minimization problem with constraints C0 and C0+C1
    '''

    K_3body = N-G+1 # Number of 3-body subsystems
    P_3body = 4**G-1 # Number of Pauli basis for 3-body subsystems
    ep = cp.Variable((K, P))
    ep_C1 = cp.Variable((N-G+1, 4**G-1))

    # Define SDP variables
    dm_tilde = []
    for k in range(K):
        dm_tilde.append( np.array(tensor([qeye(2)] * M)) / 2 **M )
    for k in range(K):
        for p in range(P):
            dm_tilde[k] = dm_tilde[k] + cp.multiply(ep[k, p], np.array(pauliToMatrix(PauliStrList_part[p])))
    dm_tilde_C1 = []
    for k in range(K_3body):
        dm_tilde_C1.append( np.array(tensor([qeye(2)] * G)) / 2 ** G )
    for k in range(K_3body): 
        for p in range(P_3body):
            dm_tilde_C1[k] = dm_tilde_C1[k] + cp.multiply(ep_C1[k, p], np.array(pauliToMatrix(PauliStrList_Gbody[p])))


    # Define constraints
    constraints_C0 = []
    for i in range(K):  # non-negative eigenvalues
        constraints_C0 += [dm_tilde[i] >> 1e-8]
    for i in range(K - 1):  # physically compatitble
        constraints_C0 += [cp.partial_trace(dm_tilde[i], dims=[2] * M, axis=0) ==
                        cp.partial_trace(dm_tilde[i + 1], dims=[2] * M, axis=M - 1)]

    constraints_C1 = []
    for i in range(K_3body): 
        constraints_C1 += [dm_tilde_C1[i] >> 1e-8]  # non-negative eigenvalues
    for i in range(K_3body):
        constraints_C1 += [cp.partial_trace(dm_tilde_C1[i], dims=[4,2], axis=1) == dm_tilde[i]]
        constraints_C1 += [cp.partial_trace(dm_tilde_C1[i], dims=[2,4], axis=0) == dm_tilde[i+1]]
    
    constraints_WM = []
    # for i in range(K): # non-negative eigenvalues
    #     constraints_WM += [dm_tilde[i] >> 1e-8]  
    #constraints_WM = [cp.von_neumann_entr(dm_tilde[0]) >= 0.75]

    # constraints_WM = [cp.von_neumann_entr(dm_tilde[0]) - cp.von_neumann_entr(cp.partial_trace(dm_tilde[0], dims=[2] * M, axis=0)) >= 0]
    
    for i in range(K_3body):
        constraints_WM += [cp.von_neumann_entr(dm_tilde[i]) -
                           cp.von_neumann_entr(cp.partial_trace(dm_tilde[i], dims=[2] * M, axis=0)) 
                           >= 0]

        # constraints_WM += [cp.von_neumann_entr(dm_tilde[i]) -
        #                    cp.von_neumann_entr(cp.partial_trace(dm_tilde[i], dims=[2] * M, axis=0))
        #                    >= 0]
        
        # constraints_WM += [cp.von_neumann_entr(dm_tilde[i])+cp.von_neumann_entr(dm_tilde[i+1]) >= 
        #                 cp.von_neumann_entr(cp.partial_trace(dm_tilde[i], dims=[2] * M, axis=M-1))+
        #                 cp.von_neumann_entr(cp.partial_trace(dm_tilde[i+1], dims=[2] * M, axis=0))]


    # Solve SDP with conditions C0
    H_exp0 = 0
    for i in range(K):
        H_exp0 = H_exp0 + H @ dm_tilde[i]
    prob_C0 = cp.Problem(
        cp.Minimize(
            cp.real(
                cp.trace(
                    H_exp0
                )
            )
        ), constraints_C0
    )
    energy_C0 = prob_C0.solve(solver=cp.SCS, verbose=False)

    # Solve SDP with conditions C1+C0
    H_exp01 = 0
    for i in range(K):
        H_exp01 = H_exp01 + H @ dm_tilde[i]
    prob_C01 = cp.Problem(
        cp.Minimize(
            cp.real(
                cp.trace(
                    H_exp01
                )
            )
        ), constraints_C0 + constraints_C1
    )
    energy_C01 = prob_C01.solve(solver=cp.SCS, verbose=False)

    # Solve SDP with WM
    H_exp_WM = 0
    for i in range(K):
        H_exp_WM = H_exp_WM + H @ dm_tilde[i]
    prob_WM = cp.Problem(
        cp.Minimize(
            cp.real(
                cp.trace(
                    H_exp_WM
                )
            )
        ), constraints_WM + constraints_C0
    )
    energy_WM = prob_WM.solve(solver=cp.SCS, verbose=False)

    return energy_C0, energy_C01, energy_WM

In [76]:
# For testing weak monotonocity constraints

N = 3 # Number of qubits of the entire system
M = 2 # Number of qubits of subsystems
G = 3 # Number of qubits of partial global system (C1)
K = N-M+1 # Number of subsystems
P = 4**M-1 # Number of Pauli basis for each subsystem

PauliStrList = generate_PauliStrList(N)[1:]
PauliStrList_part = generate_PauliStrList(M)[1:]
PauliStrList_Gbody = generate_PauliStrList(G)[1:]

H_local_list = ['II', 'XX', 'YY', 'ZZ'] # Pauli string representation of the local Hamiltonian of subsystems
H_sign_list = [1, -1, -1, -1]
H_global_list = Hamiltonian_global(H_local_list, N, M, K) # Pauli string representation of the Hamiltonian of the whole system
H_local = -1/8 * np.array( Hamiltonian_matrix(H_local_list, H_sign_list) ) # Matrix representation of the local Hamiltonian of subsystems
H_global = -1/8* np.array( Hamiltonian_matrix(H_global_list, H_sign_list*2) ) # Matrix representation of the Hamiltonian of the whole system

ground_state_energy, ground_state_dm = ground_state(H_global) 
q_state = DensityMatrix(ground_state_dm) 
lower_bound_with_SDP(H_local, N, M, K, P)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
0.0 <= von_neumann_entr([[0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.25+0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.  +0.j 0.25+0.j]] + Promote(var202958[0, 0], (4, 4)) @ [[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]] + Promote(var202958[0, 1], (4, 4)) @ [[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]] + Promote(var202958[0, 2], (4, 4)) @ [[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]] + Promote(var202958[0, 3], (4, 4)) @ [[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]] + Promote(var202958[0, 4], (4, 4)) @ [[0.+0.j 0.+0.j 0.+0

In [ ]:
dm = np.array([[1,0,0,0],[0,1,0,0]])

In [ ]:
X = 

In [23]:
"""
Program- 1
"""
X = cp.Variable(shape = (4,4), PSD = True)
objective = cp.Minimize( cp.von_neumann_entr(X) )

#objective = cp.von_neumann_entr(X)
cons1 = cp.trace(X) == 1
cp.von_neumann_entr(X)

# cons2 = cp.trace(A2 @ X) == b[1]
# cons3 = X - cp.diag(cp.diag(X)) == 0 # equivalent to 'diag=True'

prob = cp.Problem(objective, [cons1])
prob.solve(solver=cp.SCS, verbose=False)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to minimize a function that is concave.

In [15]:
prob

Problem(Minimize(Expression(AFFINE, NONNEGATIVE, ())), [Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ()))])

In [16]:
print(X.value)

None


In [ ]:
"""
Program- 1
"""
X = cp.Variable(shape = (3,3), PSD = True)
objective = cp.Minimize(-cp.von_neumann_entr(X))
cons1 = cp.trace(A1 @ X) == b[0]
cons2 = cp.trace(A2 @ X) == b[1]
cons3 = X - cp.diag(cp.diag(X)) == 0 # equivalent to 'diag=True'
prob = cp.Problem(objective, [cons1, cons2, cons3])

"""
Program-2
"""
X = cp.Variable(shape = (3,3), diag = True)
# following objective is equivalent to '-von_neumann_entr'
# in the case of X \in diag(v)
objective = cp.Minimize(-cp.sum(cp.entr(cp.diag(X))))
cons1 = cp.trace(A1 @ X) == b[0]
cons2 = cp.trace(A2 @ X) == b[1]
cons3 = X >> 0
prob = cp.Problem(objective, [cons1, cons2, cons3])